# N-Queens MCMC

## Problem statement.

What is the number of configurations of $N$ queens on an $N\times N$ chessboard such that no two queens threaten each other?

## Solution strategy.

We approach the problem in the following manner.

### Overview

We aim to construct a probability distribution $\pi$ over $S$: the configurations of queens with only diagonal threats, if they exist. We set the probability of a configuration to be low for highly conflicting queens, and high for little-conflicting queens. Finding solutions is then a matter of sampling from $\pi$.

Since the state space is large ($|S|=N!$), we employ the Metropolis-Hastings algorithm to approximate the true probability distribution $\pi$, which can be viewed as the unique stationary distribution of a Markov Chain with state space $S$ and transition matrix $P$. A state transition swaps two rows of the chessboard, or does nothing.

### Loss function
The state space of diagonally-conflicting queens $S$ is in bijection with the set of permutations of $N$ integers.

In [1]:
# Number each queen 1...N according to her column. Let them index an array z of length N that records each queen's row. 

#############
#Q1|--|--|--#
#--|--|Q3|--#
#--|--|--|Q4#
#--|Q2|--|--#
############# <=> z = [1,4,2,3]. 1 threat.

# Swap board rows 1 & 2 / Swap array elements 1 & 2 (not indices!!)

#############
#--|--|Q3|--#
#Q1|--|--|--#
#--|--|--|Q4#
#--|Q2|--|--#
############# <=> z = [2,4,1,3]. 0 threats.

If two queens $i, j$ are threatening each other on the chess board, then the following identity holds. Without loss of generality, set $j>i$. 
$$\text{col}(j) - \text{col}(i) = |\text{row}(j) - \text{row}(i)|$$

+ We numbered each queen according to her column, so $\text{col}(i)=i$.
+ The array $z$ records the rows of the queens, so $\text{row}(i)=z_i$.

Therefore, the number of threats on the board under configuration $z$ is a loss function 

$$\mathcal{L(z)}=\sum_{1\leq i<j\leq N} \mathbb{1}\{j-i=z_j-z_i\}$$

Which we aim to minimize, and can be computed in $\mathcal{O}$$N\choose 2$ operations.

### Algorithm

#### Definition

+ Input: $N>3$, the number of queens and side-length of the chess-board.
+ Output: A stream of better and better configurations of queens.

#### Notation

Let $\pi_\beta(z)=e^{-\beta\mathcal{L}(z)} / Z_\beta$ over $S$ be the probability distribution we wish to sample, where $\beta>0$ adjusts explore/exploit tradeoff of the sampling process, and $Z_\beta=\sum_{x\in S}e^{-\beta\mathcal{L}(x)}$ is a normalization constant.

Let $\Psi$ be the "base chain" which represents a symmetric random walk on $S$. Since there are $n\choose 2$ possible pair-wise swap operations, 
$$\psi_{xy}=\begin{cases}{n \choose 2}^{-1} & \text{if } d(x,y)=1\\0 &\text{otherwise}\end{cases},$$
where $d(x,y)$ is the minimum number of pairwise swaps required to configure $x$ into $y$. This chain is irreducible, positive-recurrent, period 2, and symmetric.

Let $A$ be the matrix of "acceptance probabilities"
$$a_{xy}=\min\{1,\frac{\pi_\beta(y)\psi_{yx}}{\pi_\beta(x)\psi_{xy}}\}=\min\{1,e^{-\beta(\mathcal{L}(y)-\mathcal{L}(x))}\}.$$

The $\pi_\beta$ is the left-eigenvector of the transition matrix $P$, where for $d(x,y)=1$, 
$$p_{xy} = \begin{cases}{n \choose 2}^{-1}e^{-\beta(\mathcal{L}(y)-\mathcal{L}(x))}& \text{ when } \mathcal{L}(y) > \mathcal{L}(x) \\{n \choose 2}^{-1}& \text{ when } \mathcal{L}(y) \leq \mathcal{L}(x)\end{cases},$$
and for $x=y$, 
$$p_{xx}=1-\sum_{d(x,u)=1}p_{x,u},$$
and 0 otherwise.

#### Steps

1. Let $Z_t$ be a random variable that takes values in $S$. 
2. Set $Z_0$ to the identity permutation $[1234]$ (diagonal queens).
3. Until a threat-free configuration is found, let $Z_t$ evolve according to $P$.

#### Runtime analysis

(We should do more math for $\beta(t)$, and the spectral gap)

# Implementation

#### TODO
1. Figure out source of slowdown. 
2. Investigate better expressions for beta. Perhaps adaptability?
3. Mini-batching? More randomness. No convexity -> probably not a good idea.
4. Note - max(P) always grows, where everything else is negligeable. Therefore, when nearing convergence, switch to early stopping

#### DONE
1. Instead of recalculating loss, calculate difference in loss.

In [2]:
import time, sys
from IPython.display import clear_output
import numpy as np
import itertools as it
import math

In [3]:
### Initialisation
N = 30
C = math.comb(N,2)
z0 = np.arange(1,N+1)
beta = 1

idx_pairs = np.array(list(it.combinations(z0,2))) - [1,1]
col_diff = np.array([j-i for (i,j) in idx_pairs])

In [4]:
def swap(z, i, j):
    """
    Swaps the elements of z at indices i and j, then returns z. Inplace.
    """
    z[[i, j]] = z[j], z[i]
    return z

In [5]:
### Loss function runs in n(n+1)/2 steps.
def loss(z):
    """
    Interprets z as chessboard with N queens threatening each other diagonally.
    Counts the number of unique pairs of threatening queens.
    """
    # Compute pairwise differences in z.
    row_diff = np.array([abs(z[j]-z[i]) for (i,j) in idx_pairs])
    loss = np.sum(col_diff==row_diff)
    return loss

In [6]:
### Threat count
def threats(z, i):
    """
    Returns number of queens threatening queen i.
    """
    return np.sum([abs(k-i)==abs(z[k]-z[i]) for k in range(N) if k != i])

In [7]:
### One-step loss difference runs in 4n steps.
def loss_diff(z, i, j):
    """
    Given a state z and swap operation (i,j), calculates the change in loss.
    """
    old = threats(z,i) + threats(z,j)
    y = swap(z.copy(), i, j)
    new = threats(y,i) + threats(y,j)
    return new - old

In [8]:
### Run search
MAX_ITERS=100000
z = z0.copy()
P = [0]
I = np.append(idx_pairs, [[0, 0]],axis=0)
for t in range(1, MAX_ITERS):
    
    # Calculate loss
    l = loss(z)
    
    # Print current loss
    clear_output(wait = True)
    print("t =", t, "| conflicts =", l)
    print(min(P), max(P))
    
    # If a solution is found, exit.
    if l == 0:
        break
        
    # Otherwise, evolve z.
    else:
        # Outgoing arrows. This step takes 2*(N+1)*N^2* operations, which is better than O(N!)
        P = np.array([min(1, np.exp(-np.log(t**2)*loss_diff(z,i,j))) / C for (i,j) in I])

        # Self loop
        P[-1] = max(0, 1 - sum(P[:-1]))

        # Choose a good z
        i, j = I[np.random.choice(C+1, size=1, p=P)][0]
        z = swap(z, i, j)
        
        
        
        
if (loss(z) == 0):    
    print("Here's a valid solution: ", z, "\nFound after ", t, " steps. (beta = ", np.log(t), ")")

t = 1060 | conflicts = 0
1.2958392849326461e-33 0.9586205748641892
Here's a valid solution:  [ 2 22 12  9 17 14 21  8  3 25  6 29 16 24 28 23  1  5 18 26 13 11 27 20
  7  4 15 30 19 10] 
Found after  1060  steps. (beta =  6.966024187106113 )
